In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

## Loading The Data 

- Dropping column "id" as it can be used to explicitly identify the sample. 
- Dropping column "name" as there are several non-ascii characters, plus this column would not really contribute to predicting popularity since it is simply the name of the song. 
- Dropping column "release-date" because column "year" is a cleaned up version of the same data.
- Choosing data released in 2018 onwards to reduce the massive size of the dataset. I'm also more interested in recent music. 

In [2]:
csv_url = 'https://raw.githubusercontent.com/vkashyap08/Predicting-Spotify-Song-Popularity-/main/data.csv'
original = pd.read_csv(csv_url)

In [3]:
data = original.drop(["id", "release_date", 'name'], axis=1)
data = data[(data['year']>2018)]

## Dividing feature columns into categorical and numerical for encoding

In [4]:
categorical_columns = ['artists']
numerical_columns = ['acousticness', 'danceability', 'duration_ms', 'energy', 
                     'explicit', 'instrumentalness', 'key',
                     'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence', 'year']

X = data[categorical_columns + numerical_columns]
y = data["popularity"]
y = y.values.ravel()

## 80/20 Train-Test split and 80/20 Train-Validation split

In [5]:
# splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

# splitting train into train and validation 
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size= 0.2)

## Creating Initial Pipe with Dummy Estimator

I ran the one categorical column "artists" through OneHotEncoder so that a float value could be assigned to each artist name. I ran the numerical columns through SimpleImputer so that any missing values would be fit with the mean.

In [6]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [7]:
pipe = Pipeline([
    ('preprocess', ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
                                      ('num', SimpleImputer(strategy='mean'), numerical_columns)])),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', DummyEstimator())
])

## Randomized Search CV to Hypertune Parameters

I chose to focus on Random Forest Regressor, Decision Tree Regressor, and a basic Linear Regression since the goal is to develop a model that predicts the popularity (measured on a scale of 1 to 100) of a track on Spotify, which is a regression problem. I chose a relatively big range for n_estimators for Random Forest to increase the chance of a good accuracy. Through this randomized cross validation search, I'll hopefully be able to find the best model that will give me the lowest evaluation metrics.  

In [8]:
search_space = [{'clf': [RandomForestRegressor()], 
                 'clf__n_estimators': range(1,100),
                 'clf__max_features': ['sqrt', 'log2', 'auto'],
                 'clf__min_samples_leaf': range(1,6),
                 'clf__min_samples_split': range(1,6)},
        
                {'clf': [DecisionTreeRegressor()],
                 'clf__max_depth': range(1,10),
                 'clf__max_features': [None, 'sqrt', 'auto', 'log2', 0.3, 0.5, 0.7],
                 'clf__min_samples_leaf': range(1,6),
                 'clf__criterion': ["gini", "entropy"]},
                
                {'clf': [LinearRegression()]}]

# Random Search
random_search = RandomizedSearchCV(estimator=pipe, 
                                    param_distributions=search_space, 
                                    n_iter=50,
                                    cv=3, 
                                    n_jobs=-1,
                                    verbose=1)
#  Fit grid search
best_model = random_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   37.6s finished


In [9]:
best_model.best_estimator_.get_params()['clf']

RandomForestRegressor(min_samples_split=3, n_estimators=99)

Randomized Search CV said that the hyperparams I should hypertune are:
- n_estimators=99
- min_samples_split=3

### Random Forest Default Hyperparameters

In [10]:
rf = RandomForestRegressor()
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Final Tuned Hyperparameters List

In [11]:
hyperparameters = {
 'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 3, # changed
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 99, # changed
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Fitting Final Model

In [12]:
pipe = Pipeline([
    ('preprocess', ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
                                      ('num', SimpleImputer(strategy='mean'), numerical_columns)])),
    ('scaler', StandardScaler(with_mean=False)),
    ('classifier', RandomForestRegressor(**hyperparameters))
])
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['artists']),
                                                 ('num', SimpleImputer(),
                                                  ['acousticness',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'explicit',
                                                   'instrumentalness', 'key',
                                                   'liveness', 'loudness',
                                                   'mode', 'speechiness',
                                                   'tempo', 'valence',
                                                   'year'])])),
                ('scaler', StandardScaler(with_me

## Evaluation Metrics for Model Accuracy

- RF train/test accuracy
- MSE (Mean Squared Error)
- MAE (Mean Absolute Error)

A combination of these 3 metrics will clearly tell me whether my model accurately predicted the popularity of each song. MAE measures the absolute average distance between the real data and the predicted data but fails to punish large errors in prediction. MSE measures the squared average distance between the real data and the predicted data and takes into account larger errors. Although it's hard to compare the two, I would ideally have low numbers for both. I did not choose metrics such as precision score, AUC, and F1 as they are for classification and this is a regression problem.

### RF Accuracy for Train, Validation, Test

In [13]:
print("RF train accuracy: %0.3f" % pipe.score(X_train, y_train))
print("RF validation accuracy: %0.3f" % pipe.score(X_validation, y_validation))
print("RF test accuracy: %0.3f" % pipe.score(X_test, y_test))

RF train accuracy: 0.947
RF validation accuracy: 0.601
RF test accuracy: 0.644


### MSE and MAE for Validation

In [14]:
y_pred_val = pipe.predict(X_validation)
mse = mean_squared_error(y_validation, y_pred_val)
mae = mean_absolute_error(y_validation, y_pred_val)
print(f"Mean squared error: {mse:,.2f}")
print(f"Mean absolute error: {mae:,.2f}")

Mean squared error: 350.55
Mean absolute error: 10.60


### MSE and MAE for Test

In [15]:
y_pred_test = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f"Mean squared error: {mse:,.2f}")
print(f"Mean absolute error: {mae:,.2f}")

Mean squared error: 334.14
Mean absolute error: 10.65


The train accuracy is high and leads me to believe the data has been overfit on the training dataset since the test accuracy is much lower than I would have predicted. Below is a dataframe I created which puts the actual y_test values and the predicted y_pred values side by side to compare. We can see that for example rows 1, 3, and 4 are pretty similar.

In [16]:
df = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred_test})
df.head()

,y_test,y_pred
0,68,46.240019
1,33,31.092761
2,9,1.080808
3,1,3.515404
4,0,0.000000


## Quick Comparison with default Random Forest, default Decision Tree, and default Linear Regression

### Default Random Forest

In [17]:
pipe = Pipeline([
    ('preprocess', ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
                                      ('num', SimpleImputer(strategy='mean'), numerical_columns)])),
    ('scaler', StandardScaler(with_mean=False)),
    ('rf', RandomForestRegressor()),])
pipe.fit(X_train, y_train)
y_pred_rf = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
print(f"Mean squared error: {mse:,.2f}")
print(f"Mean absolute error: {mae:,.2f}")

Mean squared error: 327.94
Mean absolute error: 10.57


### Default Decision Tree

In [18]:
pipe = Pipeline([
    ('preprocess', ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
                                      ('num', SimpleImputer(strategy='mean'), numerical_columns)])),
    ('scaler', StandardScaler(with_mean=False)),
    ('dt', DecisionTreeRegressor())])
pipe.fit(X_train, y_train)
y_pred_dt = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred_dt)
mae = mean_absolute_error(y_test, y_pred_dt)
print(f"Mean squared error: {mse:,.2f}")
print(f"Mean absolute error: {mae:,.2f}")

Mean squared error: 571.64
Mean absolute error: 11.38


### Default Linear Regression

In [19]:
pipe = Pipeline([
    ('preprocess', ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
                                      ('num', SimpleImputer(strategy='mean'), numerical_columns)])),
    ('scaler', StandardScaler(with_mean=False)),
    ('lr', LinearRegression())])
pipe.fit(X_train, y_train)
y_pred_lr = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred_lr)
mae = mean_absolute_error(y_test, y_pred_lr)
print(f"Mean squared error: {mse:,.2f}")
print(f"Mean absolute error: {mae:,.2f}")

Mean squared error: 420.46
Mean absolute error: 12.63


## Chart Detailing Evaluation Metrics of all Models

In [20]:
mae_df = pd.DataFrame(np.array([['Tuned Random Forest', '10.65', '334.14'], 
                                ['Default Random Forest', '10.57', '327.94'], 
                                ['Default Decision Tree', '11.38', '571.64'],
                                ['Default Linear Regression', '12.63', '420.46']]),
                   columns=['Model', 'MAE', 'MSE'])
mae_df

,Model,MAE,MSE
0,Tuned Random Forest,10.65,334.14
1,Default Random Forest,10.57,327.94
2,Default Decision Tree,11.38,571.64
3,Default Linear Regression,12.63,420.46


We can see from the above that surprisingly the default Random Forest actually performed the best with both a lower MSE and lower MAE, followed by tuned Random Forest, default Linear Regression as third, and default Decision Tree Regressor in last place. Overall, the MAEs were smaller than I expected which is great, but the MSEs were pretty large in general (not so great). The fact that default RF performed the best shows that a Random Forest model is very versatile for different kinds of data. 

## Summary

In summary, my evaluation metrics unfortunately were much higher than I hoped. I do not think this model is very useful. One thing I would change would be to do a more robust Randomized Search or Grid Search that would include more variety in models and more parameter options for each mode. Additionally, I believe further EDA was needed in order to really lock down which features were truly important to the prediction of popularity, rather than using almost all the columns in the dataset. In a future iteration of this project, I would use the "permutation importance" sklearn package first to choose the best features for my model. It would also be interesting to see how feature importance changed over the years. Using features such as "danceability", "energy", etc to predict a song's popularity could also be crucial to creating a more personalized user experience for music streaming giants like Spotify and Pandora. 

## Citations

https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks

https://scikit-learn.org/stable/auto_examples//inspection/plot_permutation_importance.html